# Project Code


## The following code connects with the COSMIC API and pulls genomic cancer data

In [ ]:
import requests

# Define the API endpoint
url = "https://clinicaltables.nlm.nih.gov/api/cosmic/v3/search"

gene_name = "KRAS"

# Example parameters (adjust as needed)
params = {
    "terms": gene_name,  # Query mutations in a specific gene    # Limit the number of results
    "maxList":  700,
    "grchv": 38,
    "df": "PrimarySite,MutationAA,MutationDescription"
}

# Make the GET request
response = requests.get(url, params=params)

# Check the response
if response.status_code == 200:
    data = response.json()
else:
    print(f"Error: {response.status_code}, {response.text}")
    


data = data[3:][0]


mutation_wanted = []

for val in data:
    organ, position, mut_type = val[0], val[1], val[2]
    print(position)
    if "Substitution" in mut_type and "?" not in position and "*" not in position and "X" not in position:
        mutation_wanted.append(position[2:])
        
print(len(mutation_wanted))



## Visualizes Wild Type Protein from UniProt Accession

In [ ]:
import requests
import json
import py3Dmol
from IPython.display import display, Image

def parse_mutation(mutation):
    """
    Parse a mutation string like K1A into its components.
    """
    original_aa = mutation[0]
    position = int(mutation[1:-1])  # Extract the numeric part
    new_aa = mutation[-1]
    return original_aa, position, new_aa

def fetch_uniprot_accession(gene_name):
    """
    Fetch the UniProt accession number for a given gene name.
    """
    uniprot_url = f"https://rest.uniprot.org/uniprotkb/search"
    params = {
        "query": f"gene:{gene_name}",  # Search by gene name
        "fields": "accession",        # Request only the accession field
        "format": "json",             # Get the results in JSON format
        "size": 1                     # Limit results to 1 (best match)
    }
    
    response = requests.get(uniprot_url, params=params)
    if response.status_code != 200:
        raise ValueError(f"Failed to fetch UniProt data for {gene_name}: {response.text}")
    
    # Parse the response
    results = response.json().get('results', [])
    if not results:
        raise ValueError(f"No UniProt accession found for gene {gene_name}.")
    
    # Extract the accession number from the first result
    uniprot_accession = results[0]['primaryAccession']
    return uniprot_accession


###Adapted from AFDB_API.ipynb
def visualize_protein(uniprot_accession, color="lDDT"):
    # Function to retrieve protein information by UniProt accession
    def get_protein(uniprot_accession):
        api_endpoint = "https://alphafold.ebi.ac.uk/api/prediction/"
        url = f"{api_endpoint}{uniprot_accession}"  # Construct the URL for API

        try:
            # Use a timeout to handle potential connection issues
            response = requests.get(url, timeout=10)

            # Check if the request was successful (status code 200)
            if response.status_code == 200:
                result = response.json()
                return result
            else:
                # Raise an exception for better error handling
                response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Error: {e}")

    # Function to show protein structure and image
    def show_structure_and_image(pdb_url, image_url, color):
        # Retrieve the PDB data from the URL
        pdb_data = requests.get(pdb_url).text

        # Create a 3Dmol.js view
        view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)

        # Add the PDB data to the view
        view.addModel(pdb_data, 'pdb')

        # Set style based on color parameter
        if color == "lDDT":
            view.setStyle({'cartoon': {'colorscheme': {'prop': 'b', 'gradient': 'roygb', 'min': 50, 'max': 90}}})
        elif color == "rainbow":
            view.setStyle({'cartoon': {'color': 'spectrum'}})

        # Zoom to the structure
        view.zoomTo()

        # Display the 3D structure
        display(view)

        # Retrieve and display the PNG image
        image_data = requests.get(image_url).content
        display(Image(data=image_data))


######
    protein_info = get_protein(uniprot_accession)

    if protein_info:
        # Use json.dumps with indent parameter to print formatted JSON
        print(json.dumps(protein_info, indent=2))

        # Extract PDB and PNG URLs
        pdb_url = protein_info[0].get('pdbUrl')
        image_url = protein_info[0].get('paeImageUrl')

        if pdb_url and image_url:
            # Show the protein structure and PNG image side by side
            show_structure_and_image(pdb_url, image_url, color)
        else:
            print("Failed to retrieve PDB or PNG URLs.")
    else:
        print("Failed to retrieve protein information.")

# Example pipeline
def process_mutation(gene_name, mutation):
    # Step 1: Parse the mutation
    original_aa, position, new_aa = parse_mutation(mutation)
    
    # Step 2: Fetch the protein sequence
    uniprot_accession = fetch_uniprot_accession(gene_name)
    #print(f"Original sequence: {sequence[:50]}...")  # Print first 50 residues
    
    # Step 3: Apply the mutation
    #mutated_sequence = apply_mutation(sequence, original_aa, position, new_aa)
    #print(f"Mutated sequence: {mutated_sequence[:50]}...")
    
    # Step 4: Predict the 3D structure
    #predicted_structure = submit_to_alphafold(mutated_sequence)
    
    # Step 5: Visualize the structure
    visualize_protein(uniprot_accession)

# Example Usage
gene_name = "KRAS"  # Replace with your gene name
#mutation = "K1A"    # Example mutation
#try:
    #process_mutation(gene_name, mutation)
#except Exception as e:
   # print(f"Error: {e}")
    
print(process_mutation("KRAS", "a4f"))


In [ ]:
def parse_mutation(mutation):
    """
    Parse a mutation string like K1A into its components.
    """
    original_aa = mutation[0]
    position = int(mutation[1:-1])  # Extract the numeric part
    new_aa = mutation[-1]
    return original_aa, position, new_aa

def fetch_protein_sequence(gene_name):
    """
    Fetch the protein sequence for a given gene name.
    (Here, using UniProt API as an example).
    """
    uniprot_url = f"https://rest.uniprot.org/uniprotkb/search?query=gene:{gene_name}&format=fasta"
    response = requests.get(uniprot_url)
    if response.status_code != 200:
        raise ValueError(f"Failed to fetch sequence for {gene_name}: {response.text}")
    
    # Parse FASTA response to get the sequence
    fasta_data = response.text
    sequence = "".join(line.strip() for line in fasta_data.splitlines() if not line.startswith(">"))
    return sequence

def apply_mutation(sequence, original_aa, position, new_aa):
    """
    Apply the mutation to the protein sequence.
    """
    if sequence[position - 1] != original_aa:
        raise ValueError(
            f"Original amino acid at position {position} does not match: expected {original_aa}, found {sequence[position - 1]}"
        )
    
    # Replace the original amino acid with the new one
    mutated_sequence = sequence[:position - 1] + new_aa + sequence[position:]
    return mutated_sequence

In [ ]:
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio.Align import substitution_matrices

# Define accessible surface area (ASA) values for amino acids
ASA_VALUES = {
    'A': 121, 'C': 148, 'D': 187, 'E': 214, 'F': 228, 'G': 97, 'H': 216, 
    'I': 195, 'K': 230, 'L': 191, 'M': 203, 'N': 187, 'P': 154, 'Q': 214, 
    'R': 265, 'S': 143, 'T': 163, 'V': 165, 'W': 264, 'Y': 255
}

def get_biochemical_features(sequence, mutation):
    """
    Extract biochemical feature differences for a given sequence and mutation.
    
    Parameters:
    - sequence (str): Wild-type protein sequence.
    - mutation (str): Mutation in the format [original AA][position][new AA], e.g., K1A.
    
    Returns:
    - dict: Biochemical and sequence-based feature differences.
    """
    features = {}
    
    # Parse mutation
    original_aa = mutation[0].upper()
    position = int(mutation[1:-1]) - 1  # 1-based to 0-based index
    new_aa = mutation[-1].upper()

    if position < 0 or position >= len(sequence):
        raise ValueError(f"Position {position + 1} is out of bounds for the sequence.")
    
   # if original_aa != sequence[position]:
   #    raise ValueError(f"Original amino acid {original_aa} does not match the sequence at position {position + 1}.")
    
    # Wild-type sequence properties
    seq_analysis = ProteinAnalysis(sequence)
    features["molecular_weight_diff"] = (
        ProteinAnalysis(sequence[:position] + new_aa + sequence[position + 1:]).molecular_weight()
        - seq_analysis.molecular_weight()
    )
    features["aromaticity_diff"] = (
        ProteinAnalysis(sequence[:position] + new_aa + sequence[position + 1:]).aromaticity()
        - seq_analysis.aromaticity()
    )
    features["instability_index_diff"] = (
        ProteinAnalysis(sequence[:position] + new_aa + sequence[position + 1:]).instability_index()
        - seq_analysis.instability_index()
    )
    features["isoelectric_point_diff"] = (
        ProteinAnalysis(sequence[:position] + new_aa + sequence[position + 1:]).isoelectric_point()
        - seq_analysis.isoelectric_point()
    )
    
    # Mutation-specific properties
    features["mutation_position"] = position + 1
    features["original_aa"] = original_aa
    features["new_aa"] = new_aa
    
    # Hydrophobicity (Kyte-Doolittle scale)
    kd_scale = {
        'A': 1.8, 'C': 2.5, 'D': -3.5, 'E': -3.5, 'F': 2.8, 'G': -0.4,
        'H': -3.2, 'I': 4.5, 'K': -3.9, 'L': 3.8, 'M': 1.9, 'N': -3.5,
        'P': -1.6, 'Q': -3.5, 'R': -4.5, 'S': -0.8, 'T': -0.7, 'V': 4.2,
        'W': -0.9, 'Y': -1.3
    }
    features["hydrophobicity_diff"] = kd_scale[new_aa] - kd_scale[original_aa]
    
    # Accessible Surface Area (ASA)
    features["ASA_diff"] = ASA_VALUES.get(new_aa, 0) - ASA_VALUES.get(original_aa, 0)
    
    # BLOSUM62 Substitution Score
    blosum62 = substitution_matrices.load("BLOSUM62")
    
    features["blosum62_score"] = blosum62[original_aa, new_aa]  # Handle invalid inputs gracefully
    
    return features

# Example Usage
sequence = fetch_protein_sequence(gene_name)

features = []
for mut in mutation_wanted: 
    mutation = mut
    
    features.append(get_biochemical_features(sequence, mutation))

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

np.random.seed(6701)

df = pd.DataFrame(features)
df = df.drop("original_aa", axis = 1)
df = df.drop("new_aa", axis = 1)
damage_scores = np.loadtxt("KRAS_damage_scores.csv", delimiter=",")
damage_scores = damage_scores.transpose()
X = df.values

X_train, X_test, y_train, y_test = train_test_split(df, damage_scores, test_size=0.2, random_state=42)

# Initialize and train the Random Forest Regressor model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predict damage scores on the test set
y_pred = rf_model.predict(X_test)

# Evaluate model performance (Mean Squared Error)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Optional: Evaluate with R^2 score to see how well the model explains the variance
r2_score = rf_model.score(X_test, y_test)
print(f"R^2 Score: {r2_score}")

# Example: Display predicted vs actual values for the test set
comparison = pd.DataFrame({"Actual": y_test, "Predicted": y_pred})
print(comparison.head())

# Create a scatter plot
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, color='blue', edgecolors='k', alpha=0.7)

# Add a line for perfect predictions (y_pred = y_test)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--')

# Add labels and title
plt.xlabel('True Values (y_test)')
plt.ylabel('Predicted Values (y_pred)')
plt.title('True vs Predicted Values')

# Show the plot
plt.grid(True)
plt.show()


## The above is a regressor, the following is a classifier

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler

np.random.seed(6701)


df = pd.DataFrame(features)
df = df.drop("original_aa", axis = 1)
df = df.drop("new_aa", axis = 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df, damage_scores, test_size=0.2, random_state=42)

ros = RandomOverSampler(random_state=3)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

# Initialize the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train_resampled, y_train_resampled)

# Predict on the test set
y_pred = rf_model.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Feature Importance
feature_importance = pd.DataFrame({
    'Feature': [f'Feature_{i}' for i in range(X.shape[1])],
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("\nTop Features:\n", feature_importance.head(10))

In [ ]:
import seaborn as sns
cm = confusion_matrix(y_test, y_pred)

# Plot the confusion matrix using seaborn heatmap
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, xticklabels=['0', '1', '2', '3'], yticklabels=['0', '1', '2', '3'], linewidths=1, linecolor='black')

# Add labels and title
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix for Damage Score Classifier')

# Show the plot
plt.show()